In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import os

# Keras settings
batch_size = 32
num_classes = 10
epochs = 5
num_predictions = 20

# the data split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# defining our model 
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# define model name and file locations
model_name = 'keras_cifar10_trained_model.h5'
model_output_dir = os.path.join(os.getcwd(), 'outputs')

# initiate Adam optimizer (https://keras.io/api/optimizers/adam/)
opt = Adam(learning_rate=0.001)

# define checkpoint function to only save the model after each epoch if it is "better"
# (decided based on the validation loss function) in the output file path
if not os.path.isdir(model_output_dir):
    os.makedirs(model_output_dir)
model_path = os.path.join(model_output_dir, model_name)
checkpoint_cb = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True)

# define the loss function, optimizer and additionally tracked metrics of the model training
# (https://keras.io/api/losses/probabilistic_losses/#categoricalcrossentropy-class)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
# Get the Azure Machine Learning Tracking URI
mlflow_tracking_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri
print(mlflow_tracking_uri)

In [ ]:
import mlflow

# Configuring the tracking URI
mlflow.set_tracking_uri(mlflow_tracking_uri)

# Creating an MLflow experiment
experiment_name = 'cifar10_cnn_local'
mlflow.set_experiment(experiment_name)

In [ ]:
with mlflow.start_run() as run:
    mlflow.tensorflow.autolog()

    # train the model for a certain number of epochs
    model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_split= 0.2,
            shuffle=True,
            callbacks=[checkpoint_cb])

In [ ]:
# register model trained in last run
runs = mlflow.search_runs(experiment_names=[experiment_name], output_format='list')
run_id = runs[0].info.run_id
artifact_path = 'model'
model_version = mlflow.register_model(f"runs:/{run_id}/{artifact_path}", model_name)
print(f"model trained in run {run_id} of experiment {experiment_name} registered under name {model_version.name}")